# Testing selection strategies for EvoGFuzz

We use this notebook to test our selection strategies and compare them to the default tournament selection.

## Setup

First, we import everything neccessary for our evaluation.

In [1]:
import math
import matplotlib.pyplot as plt
from numpy import trapz
import pandas as pd
from evogfuzz.evogfuzz_class import Strategy
from evogfuzz.evogfuzz_class import EvoGFuzz
from evogfuzz.input import Input
from typing import List

from debugging_framework.benchmark.program import BenchmarkProgram
from debugging_framework.benchmark.repository import BenchmarkRepository
from debugging_benchmark.calculator.calculator import CalculatorBenchmarkRepository
from debugging_benchmark.middle.middle import MiddleBenchmarkRepository
from debugging_benchmark.expression.expression import ExpressionBenchmarkRepository
from debugging_benchmark.markup.markup import MarkupBenchmarkRepository
from debugging_benchmark.tests4py_benchmark.repository import CookieCutterBenchmarkRepository
from debugging_benchmark.tests4py_benchmark.repository import PysnooperBenchmarkRepository

tests4py :: INFO     :: Loading projects


## Build the test suite

Now we can build all test programs from the `debugging-benchmark` repository.

In [2]:
repos: List[BenchmarkRepository] = [
    PysnooperBenchmarkRepository(),
    CookieCutterBenchmarkRepository(),
    CalculatorBenchmarkRepository(),
    MiddleBenchmarkRepository(),
    ExpressionBenchmarkRepository(),
    MarkupBenchmarkRepository(),
]

subjects: List[BenchmarkProgram] = []
for repo in repos:
    for prog in repo.build():
        subjects.append(prog)

tests4py :: INFO     :: Cloning https://github.com/cool-RR/PySnooper into /tmp/pysnooper_2... 
Cloning into '/tmp/pysnooper_2'...
tests4py :: INFO     :: Resetting git at /tmp/pysnooper_2 to 814abc34a098c1b98cb327105ac396f985d2413e
tests4py :: INFO     :: Creating tmp location at /tmp/tmp_pysnooper
tests4py :: INFO     :: Copying required files to /tmp/tmp_pysnooper
tests4py :: INFO     :: Checkout buggy commit id e21a31162f4c54be693d8ca8260e42393b39abd3
tests4py :: INFO     :: Copying required files from /tmp/tmp_pysnooper
tests4py :: INFO     :: Create info file
tests4py :: INFO     :: Copying resources for pysnooper_2
tests4py :: INFO     :: Checking whether Tests4Py project
tests4py :: INFO     :: Loading projects
tests4py :: INFO     :: Checking for platform linux
tests4py :: INFO     :: Check for activated python version
tests4py :: INFO     :: Using pyenv python 3.8.4
tests4py :: INFO     :: Creating virtual env
tests4py :: INFO     :: Activating virtual env
tests4py :: INFO    

Verify, that all programs are built:

In [19]:
subjects[:2]

[Tests4PyBenchmarkProgram(pysnooper_2), Tests4PyBenchmarkProgram(pysnooper_3)]

## Running the Benchmark

We define a helper function `run_benchmark` which run the program once using the given selection strategy for the given amout of iterations. It returns the amount of exception triggering inputs found for each iteration in a numpy array. To prevent potential errors from stopping our (time consuming) benchmark, we catch any exceptions and simply return an array full of -1's to flag the failed run while other runs can keep running. 

In [8]:
import numpy as np
from evogfuzz.evogfuzz_class import Strategy
def run_benchmark(strategy:Strategy, program, iterations=100):
    try:
        epp = EvoGFuzz(
            grammar=program.get_grammar(),
            oracle=program.get_oracle(),
            inputs=program.get_passing_inputs(),
            iterations=iterations,
            strategy=strategy
            )
        found_exception_inputs = epp.fuzz()
        test = epp.get_benchmark()
        data_run = [data[1] for data in test]
    except:
        data_run = np.full(iterations, -1)
        print(f"Error: {strategy} in {program}")
    return data_run

We make use of multiprocessing to accelerate our benchmarking proccess. We decided to use `ProcessPool` from `pathos` because if allows us to pass the test programs as a parameter. The optimal amout of nodes will depend on the hardware used. We found 50 nodes to perform well on the `gruenau3` server we used.
All selection strategies are tested for the given amount of runs and iterations using the given test program. Once finished, the results are saved as a .csv file.

In [10]:
from pathos.pools import _ProcessPool as Pool
import logging
import sys
logging.disable(sys.maxsize)
from evogfuzz.evogfuzz_class import Strategy
import pandas as pd

def benchmark_program(subject, runs:int, iterations:int):
    strategies = [Strategy.RANK, Strategy.TOURNAMENT, Strategy.ROULETTE, Strategy.STOCHASTIC_UNIVERSAL_SAMPLING, Strategy.TRUNCATION]
    p = Pool(50)
    tournament = []
    roulette = []
    sus = []
    rank = []
    trunc = []
    for _ in range(runs):
        tournament.append(p.apply_async(run_benchmark, (Strategy.TOURNAMENT, subject, iterations)))
        roulette.append(p.apply_async(run_benchmark, (Strategy.ROULETTE, subject, iterations)))
        sus.append(p.apply_async(run_benchmark, (Strategy.STOCHASTIC_UNIVERSAL_SAMPLING, subject, iterations)))
        rank.append(p.apply_async(run_benchmark, (Strategy.RANK, subject, iterations)))
        trunc.append(p.apply_async(run_benchmark, (Strategy.TRUNCATION, subject, iterations)))
    p.close()
    p.join()
    tournament_res = []
    roulette_res = []
    sus_res = []
    rank_res = []
    trunc_res = []
    for a in tournament:
        tournament_res.append(a.get())
    for a in roulette:
        roulette_res.append(a.get())
    for a in sus:
        sus_res.append(a.get())
    for a in rank:
        rank_res.append(a.get())
    for a in trunc:
        trunc_res.append(a.get())
    filename = f"passing_inputs_{subject}_{Strategy.TOURNAMENT}.csv"
    df = pd.DataFrame(tournament_res)
    df.to_csv(filename)
    filename = f"passing_inputs_{subject}_{Strategy.ROULETTE}.csv"
    df = pd.DataFrame(roulette_res)
    df.to_csv(filename)
    filename = f"passing_inputs_{subject}_{Strategy.STOCHASTIC_UNIVERSAL_SAMPLING}.csv"
    df = pd.DataFrame(sus_res)
    df.to_csv(filename)
    filename = f"passing_inputs_{subject}_{Strategy.RANK}.csv"
    df = pd.DataFrame(rank_res)
    df.to_csv(filename)
    filename = f"passing_inputs_{subject}_{Strategy.TRUNCATION}.csv"
    df = pd.DataFrame(trunc_res)
    df.to_csv(filename)


## Evaluation

Finally, we can use the data we collected to calculate our performance metrics, test their statistical significance and plot them.

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from numpy import trapz
from evogfuzz.evogfuzz_class import Strategy

strategies = [Strategy.RANK, Strategy.TOURNAMENT, Strategy.ROULETTE, Strategy.STOCHASTIC_UNIVERSAL_SAMPLING, Strategy.TRUNCATION]
for subj in subjects:
    runs = 30
    # Change filenames for initial/passing inputs
    file_tournament = f"./results_passing_inputs/passing_{subj}_{Strategy.TOURNAMENT}.csv"
    file_rank = f"./results_passing_inputs/passing_{subj}_{Strategy.RANK}.csv"
    file_roulette = f"./results_passing_inputs/passing_{subj}_{Strategy.ROULETTE}.csv"
    file_sus = f"./results_passing_inputs/passing_{subj}_{Strategy.STOCHASTIC_UNIVERSAL_SAMPLING}.csv"
    file_truncation = f"./results_passing_inputs/passing_{subj}_{Strategy.TRUNCATION}.csv"
    
    df = pd.read_csv(file_tournament).to_numpy()
    df = np.delete(df, 0, 1)
    tournament = df.mean(axis=0)
    aoc_tournament = [trapz(y, dx=1) for y in df]
    total_tournament = [data[99] for data in df]
    tte_tournament = np.zeros(runs)
    for i in range(runs):
        for j in range(100):
            if df[i][j] > 0:
                tte_tournament[i] = j + 1
                break
            tte_tournament[i] = j + 1
    df = pd.read_csv(file_rank).to_numpy()
    df = np.delete(df, 0, 1)
    rank = df.mean(axis=0)
    aoc_rank = [trapz(y, dx=1) for y in df]
    total_rank = [data[99] for data in df]
    tte_rank = np.zeros(runs)
    for i in range(runs):
        for j in range(100):
            if df[i][j] > 0:
                tte_rank[i] = j + 1
                break
            tte_rank[i] = j + 1
    df = pd.read_csv(file_roulette).to_numpy()
    df = np.delete(df, 0, 1)
    roulette = df.mean(axis=0)
    aoc_roulette = [trapz(y, dx=1) for y in df]
    total_roulette = [data[99] for data in df]
    tte_roulette = np.zeros(runs)
    for i in range(runs):
        for j in range(100):
            if df[i][j] > 0:
                tte_roulette[i] = j + 1
                break
            tte_roulette[i] = j + 1
    df = pd.read_csv(file_sus).to_numpy()
    df = np.delete(df, 0, 1)
    sus = df.mean(axis=0)
    aoc_sus = [trapz(y, dx=1) for y in df]
    total_sus = [data[99] for data in df]
    tte_sus = np.zeros(runs)
    for i in range(runs):
        for j in range(100):
            if df[i][j] > 0:
                tte_sus[i] = j + 1
                break
            tte_sus[i] = j + 1
    df = pd.read_csv(file_truncation).to_numpy()
    df = np.delete(df, 0, 1)
    truncation = df.mean(axis=0)
    aoc_truncation = [trapz(y, dx=1) for y in df]
    total_truncation = [data[99] for data in df]
    tte_truncation = np.zeros(runs)
    for i in range(runs):
        for j in range(100):
            if df[i][j] > 0:
                tte_truncation[i] = j + 1
                break
            tte_truncation[i] = j + 1
    plt.figure(dpi=1200)
    plt.plot(tournament, label="Tournament")
    plt.plot(rank, label="Rank")
    plt.plot(roulette, label="Roulette")
    plt.plot(sus, label="Stochastic Universal Sampling")
    plt.plot(truncation, label="Truncation")
    
    plt.title(subj)
    plt.xlabel('Iteration')
    plt.ylabel('Found exception inputs')
    plt.legend()
    plt.ylim(bottom=0)
    plt.xlim(left=0)
    plt.savefig(f"./Passing_plots/{subj}.png", dpi=1200) # Change for initial inputs
    plt.show()
    
    print("Area under curve")
    print("Tournament: ", np.mean(aoc_tournament))
    print("Rank : ", np.mean(aoc_rank), mannwhitneyu(aoc_rank, aoc_tournament, alternative="greater").pvalue)
    print("Roulette: ", np.mean(aoc_roulette), mannwhitneyu(aoc_roulette, aoc_tournament, alternative="greater").pvalue)
    print("SUS ", np.mean(aoc_sus), mannwhitneyu(aoc_sus, aoc_tournament, alternative="greater").pvalue)
    print("Truncation: ", np.mean(aoc_truncation), mannwhitneyu(aoc_truncation, aoc_tournament, alternative="greater").pvalue)
    
    print("\nTTE")
    print("Tournament: ", np.mean(tte_tournament))
    print("Rank : ", np.mean(tte_rank), mannwhitneyu(tte_rank, tte_tournament, alternative="less").pvalue)
    print("Roulette: ", np.mean(tte_roulette), mannwhitneyu(tte_roulette, tte_tournament, alternative="less").pvalue)
    print("SUS ", np.mean(tte_sus), mannwhitneyu(tte_sus, tte_tournament, alternative="less").pvalue)
    print("Truncation: ", np.mean(tte_truncation), mannwhitneyu(tte_truncation, tte_tournament, alternative="less").pvalue)
    
    print("\nTotal exceptions")
    print("Tournament: ", np.mean(total_tournament))
    print("Rank : ", np.mean(total_rank), mannwhitneyu(total_rank, total_tournament, alternative="greater").pvalue)
    print("Roulette: ", np.mean(total_roulette), mannwhitneyu(total_roulette, total_tournament, alternative="greater").pvalue)
    print("SUS ", np.mean(total_sus), mannwhitneyu(total_sus, total_tournament, alternative="greater").pvalue)
    print("Truncation: ", np.mean(total_truncation), mannwhitneyu(total_truncation, total_tournament, alternative="greater").pvalue)
    

    


FileNotFoundError: [Errno 2] No such file or directory: './results_passing_inputs/Tests4PyBenchmarkProgram(pysnooper_2)_Tournament.csv'